In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationStore, setup_new_locations
from autodm.storyline import StoryLineWithStoryItems

from rich import print

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from datetime import datetime
from llama_index.core import PromptTemplate, StorageContext, VectorStoreIndex
from pydantic import BaseModel, Field
from typing import Optional, List, Any
from pathlib import Path
import torch

In [ ]:
character = Character.generate(level=0)
print(character)

In [ ]:
locations = setup_new_locations()
print(locations)

In [ ]:
city, region = locations.current, locations.get(locations.current.parent_name)

In [ ]:
story = StoryLineWithStoryItems.generate(character, city, region, locations)

In [ ]:
VECTORSTORE_DIR = Path('~').expanduser() / '.autodm/vectorstore'
if not VECTORSTORE_DIR.exists():
    VECTORSTORE_DIR.mkdir(parents=True)

ADVENTURE_LOG_TABLE_NAME = 'adventure_log'

In [ ]:
adventure_log_vectorstore = LanceDBVectorStore(uri=str(VECTORSTORE_DIR), table_name=str(ADVENTURE_LOG_TABLE_NAME))
storage_context = StorageContext.from_defaults(vector_store=adventure_log_vectorstore)

In [5]:
def get_device():
    if torch.backends.mps.is_available():
        return 'mps'
    elif torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [ ]:
class AdventureLog:
    def __init__(self, lance_uri=VECTORSTORE_DIR, lance_table=ADVENTURE_LOG_TABLE_NAME):
        self.lance_uri = lance_uri
        self.lance_table = lance_table
        self.storage_context = self._setup_storage_context()
        self.index = self._setup_index()

    def _setup_storage_context(self):
        return StorageContext.from_defaults(
            vector_store=LanceDBVectorStore(
                uri=str(self.lance_uri), table_name=str(self.lance_table)
            )
        )
    
    def _setup_index(self):
        index = VectorStoreIndex(
            embed_model=HuggingFaceEmbedding(
                "Alibaba-NLP/gte-base-en-v1.5", device=get_device(), trust_remote_code=True
            ),
            storage_context=self.storage_context,
        )
        return index
        
    
    def add_entry(self, entry:str):
        